In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# model path
model_path = "model/ner/1120_t5_large_diary_finetuned_epoch_3.pt"
# default tokenizer (did not customize)
tokenizer = T5Tokenizer.from_pretrained("t5-large")

# model load
model = T5ForConditionalGeneration.from_pretrained("t5-large")
model.load_state_dict(torch.load(model_path))  # weights
model.eval()

/var/folders/ld/ssbrz8sd4rdfbzqjq4zjrfq80000gn/T/ipykernel_87674/1797086192.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [4]:
def test_model(model, tokenizer, diary_entry):
    input_text = f"Extract Event, Action, Time, and Thoughts from: '{diary_entry}'"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

diary_entry = """The flight to London, once $600, skyrocketed to $3600 in just four days. I stared at the screen in disbelief, refreshing the page as if hoping it was a glitch. Planning the trip felt exciting at first, but now it seemed almost impossible. I wondered if I should wait, gamble on a price drop, or let the dream go for now. Travel, it seems, has its own way of testing patience."""
result = test_model(model, tokenizer, diary_entry)
print("Generated Output:", result)

Generated Output: "event": ["flight to London", "price drop"], "action": ["started at the screen in disbelief", "refreshing the page"], "time": ["four days"], "thoughts": ["travel has its own way of testing patience"]


In [6]:
# test prompt
diary_entry = """
I woke up to the gentle sound of dripping water. The air was heavy with humidity, clinging to my skin. The house felt unusually quiet, as if holding its breath. Stepping outside, I noticed puddles forming under a gray sky. A light rain had begun, soft and soothing, bringing a sense of calm. It felt like the world was pausing, just for a moment
"""
result = test_model(model, tokenizer, diary_entry)
print("Generated Output:", result)

Generated Output: "event": ["dripping water"], "action": ["noticed puddles forming"], "time": [], "thoughts": ["felt unusually quiet", "brought a sense of calm"]
